In [1]:
import numpy as np
import imageio
from skimage import transform
import matplotlib.pyplot as plt
from math import sqrt
import glob
import subprocess

%matplotlib inline

In [2]:
def load_image(img_path, resize=True):
    tmp_img = imageio.imread(img_path)
    if resize:
        return transform.resize(image=tmp_img,output_shape=(200,200),anti_aliasing=True, mode='constant')
    return tmp_img

In [2]:
import extract_features
%timeit feature = extract_features.get_vector_resnet50('data/images/golden1.jpg')

20 ms ± 290 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [2]:
import extract_features
%timeit feature = extract_features.get_vector_resnet18('data/images/golden1.jpg')

10.4 ms ± 65.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


**Resnet 50** takes $2$ times the amount of time as **Resnet 18**.

In [3]:
classes = sorted([path.split("/")[-1] for path in glob.glob("data/videos/*")])
print(classes)

['animals', 'children', 'christmas', 'nature', 'school', 'snow', 'work']


In [4]:
feature = extract_features.get_vector_resnet50('data/images/golden1.jpg')

In [7]:
feature.shape

(2048,)